[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AMLA-UBC/100-Exploring-the-World-of-Modern-Machine-Learning/blob/main/Intro_to_Stable_Diffusion_Tutorial.ipynb)

## Stable Diffusion (Text-to-image) with the Sygil WebUI

Sygil provies a simple WebUI for us to work with Stable Diffusion models and basic text-to-image settings right from our browser.

First, let's clone Sygil WebUI GitHub repo and install required modules.


In [ ]:
import sys, os

#@markdown
!nvidia-smi -L
!git clone https://github.com/Sygil-Dev/sygil-webui stable-diffusion
%cd /content/stable-diffusion
!pip install -r requirements.txt

# %cd /content/stable-diffusion
# !wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
# !chmod +x Miniconda3-latest-Linux-x86_64.sh
# !bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
# !rm Miniconda3-latest-Linux-x86_64.sh
sys.path.append('/usr/local/lib/python3.7/site-packages/')

## Setup Environment and Upscalers - CFPGan and R-ESRGAN

In [ ]:
#@markdown Conda environment takes roughly 15 minutes to 
# !conda env update -n base -f /content/stable-diffusion/environment.yaml 

#@markdown **GFPGAN** automatically corrects distorted faces with a built-in GFPGAN option, fixes them in less than half a second

#@markdown **RESRGAN** boosts the resolution of images

#@markdown **LDSR** and GoBig enable amazing upscale options in the new Image Lab

add_CFP = False #@param {type:"boolean"} 
add_RESRGAN = True #@param {type:"boolean"}
add_LDSR = False #@param {type:"boolean"} 

if add_CFP:
  %cd /content/stable-diffusion/src/gfpgan/
  !pip install basicsr facexlib yapf lmdb opencv-python pyyaml tb-nightly --no-deps
  !python setup.py develop
  !pip install realesrgan
  !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models
if add_RESRGAN:
  %cd /content/stable-diffusion/src/realesrgan/
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models
if add_LDSR:
  %cd /content/stable-diffusion/src
  !git clone https://github.com/devilismyfriend/latent-diffusion
  %cd latent-diffusion
  %mkdir -p experiments/
  %cd experiments/
  %mkdir -p pretrained_models
  %cd pretrained_models
  #project.yaml download
  !wget -O project.yaml https://heibox.uni-heidelberg.de/f/31a76b13ea27482981b4/?dl=1
  #model.ckpt model download
  !wget -O model.ckpt https://heibox.uni-heidelberg.de/f/578df07c8fc04ffbadf3/?dl=1

%cd /content/stable-diffusion/
!wget https://github.com/matomo-org/travis-scripts/blob/master/fonts/Arial.ttf?raw=true -O arial.ttf

### Model Checkpoint Settings

In [ ]:
print("Local Path Variables:\n")

models_path = "models" #@param {type:"string"}
output_path = "output" #@param {type:"string"}

#@markdown
model_checkpoint = "Lykon/DreamShaper" #@param @param ["Lykon/DreamShaper","claudfuen/photorealistic-fuen-v1","nitrosocke/Nitro-Diffusion","XpucT/Deliberate","runwayml/stable-diffusion-v1-5"] {allow-input: true}

URL_MAP = {
    "Lykon/DreamShaper": "https://huggingface.co/Lykon/DreamShaper/resolve/main/Dreamshaper_3.32_baked_vae_clip_fix.ckpt",
    "claudfuen/photorealistic-fuen-v1": "https://huggingface.co/claudfuen/photorealistic-fuen-v1/resolve/main/model.ckpt",
    "nitrosocke/Nitro-Diffusion" : "https://huggingface.co/nitrosocke/Nitro-Diffusion/resolve/main/nitroDiffusion-v1.ckpt",
    "XpucT/Deliberate": "https://huggingface.co/XpucT/Deliberate/resolve/main/Deliberate.safetensors",
    "runwayml/stable-diffusion-v1-5": "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt"
}

import os
os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

model_url = URL_MAP[model_checkpoint]
model_checkpoint = model_checkpoint.replace("/", "_")
model_checkpoint += ".ckpt" if ".ckpt" in model_checkpoint else ".safetensors"
if not os.path.exists(f'/content/{models_path}/{model_checkpoint}'):
    # !git lfs install --system --skip-repo
    # !mkdir sd-model
    # %cd /content/sd-model/
    # !git init
    !wget -P '/content/{models_path}' {model_url} -O '/content/{models_path}/{model_checkpoint}'
else:
    print("Model already downloaded. Moving on to the next step")

print(f"models_path: {models_path}")
print(f"output_path: {output_path}")

Local Path Variables:

Model already downloaded. Moving on to the next step
models_path: models
output_path: output


## Sygil WebUI Settings

In [ ]:
share_password="" #@param {type:"string"}
#@markdown * Add a password to access your WebUI
defaults="configs/webui/webui.yaml" #@param {type:"string"}
#@markdown * Path to configuration file providing WebUI defaults, uses same format as cli parameter.
#@markdown Edit this file if you want to change the default settings UI launches with.

#@markdown ---
save_metadata = False #@param {type:"boolean"}
#@markdown * Whether to embed the generation parameters in the sample images
skip_grid = False #@param {type:"boolean"}
#@markdown * Do not save a grid, only individual samples. Helpful when evaluating lots of samples
skip_save = False #@param {type:"boolean"}
#@markdown * Do not save individual samples as files. For speed measurements
optimized = False #@param {type:"boolean"}
#@markdown * Load the model onto the device piecemeal instead of all at once to reduce VRAM usage at the cost of performance
optimized_turbo = True #@param {type:"boolean"}
#@markdown * Alternative optimization mode that does not save as much VRAM but runs siginificantly faster
no_verify_input = False #@param {type:"boolean"}
#@markdown * Do not verify input to check if it's too long
no_half = False #@param {type:"boolean"}
#@markdown * Do not switch the model to 16-bit floats
no_progressbar_hiding = True #@param {type:"boolean"}
#@markdown * Do not hide progressbar in gradio UI
extra_models_cpu = False #@param {type:"boolean"}
#@markdown * Run extra models (GFGPAN/ESRGAN) on cpu
esrgan_cpu = True #@param {type:"boolean"}
#@markdown * run ESRGAN on cpu
gfpgan_cpu = False #@param {type:"boolean"}
#@markdown * run GFPGAN on cpu


run_string_with_variables = {
 '--save-metadata': f'{save_metadata}',
 '--skip-grid': f'{skip_grid}',
 '--skip-save': f'{skip_save}',
 '--optimized': f'{optimized}',
 '--optimized-turbo': f'{optimized_turbo}',
 '--no-verify-input': f'{no_verify_input}',
 '--no-half': f'{no_half}',
 '--no-progressbar-hiding': f'{no_progressbar_hiding}',
 '--extra-models-cpu': f'{extra_models_cpu}',
 '--esrgan-cpu': f'{esrgan_cpu}',
 '--gfpgan-cpu': f'{gfpgan_cpu}'}

only_true_vars = {k for (k,v) in run_string_with_variables.items() if v == 'True'}
vars = " ".join(only_true_vars)


## Launch WebUI for Stable Diffusion

#### **AMLA Members**

Do not change the value of `Images per batch`. Otherwise, no generated image will appear on the WebUI, although all generated images are saved to `sygil-webui/outputs` locally.

Play with the the length of the input text, `width`, `height`, `CFG`, `sampling steps`, `sampling method`, and `image-to-image` to visualize the basic functionalities of Stable Diffusion. Answer the following questions before continuing with our next lecture: 
- What image dimensions (width and height ratios) cause the output to look distorted? Why may this be the case? Recall that models based on Stable Diffusion v1.5 are often trained on 512x512 images.
- How can you avoid duplicate bodies or faces in image generation? In how many ways can you teach the diffusion model to generate higher resolution images? Use your knowledge of GANs and VAEs from previous sections of this unit.
- RunwayML trained Stable Diffusion v1.5 on a dataset of 512x512 images. How can the diffusion process still sometimes generate coherent 640x640 images?

Click the public URL to launch WebUI in another tab. 

In [ ]:
#@markdown ![](https://user-images.githubusercontent.com/463317/187105407-dd9b0f4e-c8da-49d3-8c78-1767f5c9aa83.jpg)

#fix adding share_password to the launch params, and also changin {vars} to $vars as it was causing webui.py to fail.

%cd /content/stable-diffusion

if share_password == "":
  !python scripts/webui.py \
  --ckpt '{models_path}/{model_checkpoint}' \
  --outdir '{output_path}' \
  --share $vars
else:
  !python scripts/webui.py \
  --ckpt '{models_path}/{model_checkpoint}' \
  --outdir '{output_path}' \
  --share-password '{share_password}' \
  --share $vars